### Importierte Biblotheken

In [28]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
from pathlib import Path



#### Voraussetzungen
* **Bibliotheken:** `kaggle`, `os`, `shutil` (`pip install kaggle`).
* **Kaggle API Key:** Eine `kaggle.json` API-Schlüsseldatei im Verzeichnis `~/.kaggle/` ist **zwingend erforderlich**. 
Ohne sie schlägt die Authentifizierung im Download-Schritt fehl. Details zur Einrichtung finden Sie in der [Kaggle Dokumentation](https://www.kaggle.com/docs/api) oder im Projektverzeichnis in der README.md Datei.


### Prüfung, Download 

Die nächste Code-Zelle führt die Kernlogik aus:

1.  **Prüfung:** Testet, ob der Datensatz bereits am `data_dir` existiert und Inhalt hat.
2.  **Download (falls nötig):**
    * **Kaggle API:** Die `kaggle` API wird initialisiert und authentifiziert sich über Ihre `~/.kaggle/kaggle.json`-Datei.
    * **Download & Entpacken:** `api.dataset_download_files(...)` lädt den Datensatz mit der `DATASET_ID` herunter und entpackt (`unzip=True`) dessen Inhalt direkt in den `data_dir`.


In [29]:
DATASET_ID = "thecoderenroute/instagram-posts-dataset"

base_dir = Path.cwd()
if base_dir.name == "notebooks":
    base_dir = base_dir.parent
    print(f"Aktueller Pfad: {base_dir}")

data_dir = base_dir / "daten" / "rohdaten" / "instagram-posts-dataset"
os.makedirs(data_dir, exist_ok=True)

data_exists = False
valid_files = [f for f in os.listdir(data_dir) if not f.startswith('.')]
if os.path.exists(data_dir) and valid_files:
    try:
        if len(valid_files) > 0:
             data_exists = True
             print(f"Datensatz bereits vorhanden unter: {data_dir}")
        else:
             print(f"Hinweis: Zielordner '{data_dir}' existiert, ist aber leer.")
    except OSError:
         print(f"Hinweis: Pfad '{data_dir}' existiert, aber Inhalt konnte nicht gelesen werden.")

# ___DOWNLOAD___
if not data_exists:
    print(f"Datensatz nicht oder nur leer vorhanden unter '{data_dir}'. Starte Download via Kaggle API...")
    try:
        print("Initialisiere Kaggle API...")
        api = KaggleApi()
        api.authenticate() 
        print("Authentifizierung erfolgreich.")
        print(f"Lade '{DATASET_ID}' nach '{data_dir}' herunter und entpacke...")
        api.dataset_download_files(DATASET_ID, path=data_dir, unzip=True)
        print("Download und Entpacken abgeschlossen.")

    except ImportError:
         print(f"**FEHLER:** Bibliothek 'kaggle' nicht gefunden. Bitte installieren: pip install kaggle")
    except Exception as e:
        print(f"**FEHLER** beim Download via Kaggle API: {e}")
else:
    print("Datensatz bereits vorhanden.")
for file in os.listdir(data_dir):
    print(f"Gefundene Datei: {file}")

Datensatz bereits vorhanden unter: /Users/zhanna97yan/Library/CloudStorage/OneDrive-SEPadersoftGmbH&Co.KG/Dokumente/FHDW/FHDW Projects/ADA_Instagram_Like_Vorhersage/daten/rohdaten/instagram-posts-dataset
Datensatz bereits vorhanden.
Gefundene Datei: .DS_Store
Gefundene Datei: Data
